The purpose of the code is to calculate the squares and cubes of a list of numbers in parallel, i.e., using two separate processes (p1 and p2). This leverages multiple CPU cores to perform computations simultaneously.

In [1]:
from multiprocessing import Process

# Function to calculate squares
def calculate_squares(numbers):
    for n in numbers:
        print(f"Square: {n*n} \n")

# Function to calculate cubes
def calculate_cubes(numbers):
    for n in numbers:
        print(f"Cube: {n*n*n} \n")

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    # Creating two processes
    p1 = Process(target=calculate_squares, args=(numbers,))
    #A Process object represents a separate process.
    #It takes two main arguments:
    #target: The function to be executed.
    #args: A tuple containing arguments for the target function.

    p2 = Process(target=calculate_cubes, args=(numbers,))

    p1.start() # start(): Starts the execution of a process in parallel.
    p2.start()

    p1.join() # Ensures the main process waits for the child processes (p1 and p2) to complete before proceeding.
    p2.join()

    print("Done!")


Done!


In [3]:
from multiprocessing import Process, Array
import numpy as np

# Function to multiply a row of matrix A with matrix B
def multiply_row(result, matrix_a, matrix_b, row_index):
    for col in range(len(matrix_b[0])):
        result[row_index * len(matrix_b[0]) + col] = sum(
            matrix_a[row_index][k] * matrix_b[k][col] for k in range(len(matrix_b))
        )

if __name__ == "__main__":
    # Matrices to multiply
    matrix_a = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
    matrix_b = [[9, 8, 7], [6, 5, 4], [3, 2, 1]]

    # Shared memory for storing the result
    result = Array('i', len(matrix_a) * len(matrix_b[0]))
    #Allows processes to store and share computation results.
    #Array('i', size) creates a shared integer array with size elements.


    # List to store processes
    processes = []

    # Create a process for each row of matrix A
    for row_index in range(len(matrix_a)):
        p = Process(target=multiply_row, args=(result, matrix_a, matrix_b, row_index))
        # Process: Represents a new independent process that will execute the multiply_row function.
        #result: The shared memory array to store the computed results.
        #matrix_a: The full matrix A (to access the current row).
        #matrix_b: The full matrix B (to perform the dot product with the row from matrix_a).
        #row_index: Identifies which row of matrix_a the process should work on.

        processes.append(p) #This list is used later to manage (e.g., start, join) all processes.
        p.start()

    # Wait for all processes to complete
    for p in processes:
        p.join()

    # Reshape and print the result
    result_matrix = np.array(result).reshape(len(matrix_a), len(matrix_b[0]))
    print("Resultant Matrix:")
    print(result_matrix)


Resultant Matrix:
[[ 30  24  18]
 [ 84  69  54]
 [138 114  90]]


In [4]:
from multiprocessing import Process, Array
import numpy as np

# Function to compute a single row of the resulting matrix
def multiply_row(shared_result, row_index, matrix_a, matrix_b, num_cols_b):
    """
    Multiply a single row of matrix A with all columns of matrix B.
    Store the result in the shared memory (shared_result).
    """
    for col in range(num_cols_b):
        # Dot product of row `row_index` of A and column `col` of B
        shared_result[row_index * num_cols_b + col] = sum(
            matrix_a[row_index][k] * matrix_b[k][col] for k in range(len(matrix_b))
        )

if __name__ == "__main__":
    # Generate large matrices A and B
    # Matrix A: 500x500, Matrix B: 500x500
    matrix_a = np.random.randint(0, 10, (500, 500))
    matrix_b = np.random.randint(0, 10, (500, 500))

    # Shared memory to store the resulting matrix
    result = Array('i', 500 * 500)  # 500x500 matrix, stored as a 1D array
    num_rows_a, num_cols_b = 500, 500  # Dimensions of the result matrix

    # List to hold the processes
    processes = []

    # Create and start a process for each row in matrix A
    for row_index in range(num_rows_a):
        p = Process(target=multiply_row, args=(result, row_index, matrix_a, matrix_b, num_cols_b))
        processes.append(p)
        p.start()

    # Wait for all processes to finish
    for p in processes:
        p.join()

    # Convert the shared memory back to a 2D NumPy array
    result_matrix = np.array(result[:]).reshape(num_rows_a, num_cols_b)

    # Print the result (optional, as the matrix is large)
    print("Matrix Multiplication Completed!")
    print("Result Matrix:", result_matrix)


Matrix Multiplication Completed!
Result Matrix: [[10183 11072 10525 ... 10351 11183 10628]
 [ 9764 10082 10041 ...  9534 10462  9990]
 [10434 10761 10642 ...  9837 10864 10169]
 ...
 [ 9700 10660 10507 ...  9749 10894 10592]
 [ 9857 10192 10242 ...  9548 10428 10551]
 [ 9615  9871  9583 ...  9347  9834  9805]]


In [5]:
from multiprocessing import Pool
import numpy as np
import time

# Function to compute a single row of the resulting matrix
def multiply_row(args):
    row_index, matrix_a, matrix_b = args
    num_cols_b = len(matrix_b[0])
    result_row = [
        sum(matrix_a[row_index][k] * matrix_b[k][col] for k in range(len(matrix_b)))
        for col in range(num_cols_b)
    ]
    return row_index, result_row

if __name__ == "__main__":
    # Generate large matrices A and B
    matrix_a = np.random.randint(0, 10, (500, 500))
    matrix_b = np.random.randint(0, 10, (500, 500))

    # Parallel Processing using Pool
    print("Starting Optimized Parallel Matrix Multiplication...")
    start_parallel = time.time()

    # Prepare arguments for each row
    num_rows_a = len(matrix_a)
    args = [(row_index, matrix_a, matrix_b) for row_index in range(num_rows_a)]

    # Use a pool of workers equal to the number of CPU cores
    with Pool() as pool:
        results = pool.map(multiply_row, args)

    # Combine results into the final matrix
    result_matrix_parallel = np.zeros((500, 500), dtype=int)
    for row_index, result_row in results:
        result_matrix_parallel[row_index] = result_row

    end_parallel = time.time()
    print("Parallel Matrix Multiplication Completed!")
    print(f"Parallel Execution Time: {end_parallel - start_parallel:.4f} seconds")

    # Sequential Processing for Comparison
    print("Starting Sequential Matrix Multiplication...")
    start_sequential = time.time()

    result_matrix_sequential = np.zeros((500, 500), dtype=int)
    for i in range(len(matrix_a)):
        for j in range(len(matrix_b[0])):
            result_matrix_sequential[i][j] = sum(matrix_a[i][k] * matrix_b[k][j] for k in range(len(matrix_b)))

    end_sequential = time.time()
    print("Sequential Matrix Multiplication Completed!")
    print(f"Sequential Execution Time: {end_sequential - start_sequential:.4f} seconds")

    # Verify results
    print("Comparison of Results:")
    print(f"Are results equal? {np.array_equal(result_matrix_parallel, result_matrix_sequential)}")


Starting Optimized Parallel Matrix Multiplication...
Parallel Matrix Multiplication Completed!
Parallel Execution Time: 94.8711 seconds
Starting Sequential Matrix Multiplication...
Sequential Matrix Multiplication Completed!
Sequential Execution Time: 95.7930 seconds
Comparison of Results:
Are results equal? True


Distributed Computing